In [142]:
# Change directory to level up to import functions
import os
os.chdir('..')

In [146]:
import pandas as pd

In [217]:
def create_piecewise_regression_model(df, position, season):
    df_subset = df[(df['position'] == position) & (df['season'] == season)]

    # Specify the feature columns explicitly, including GK
    feature_columns = [
        'value_first_gw', 
        'team_strength']

    # Step 2: Split the data into features (X) and target (y)
    X = df_subset[feature_columns]
    y = df_subset['total_points']

    # Create a linear regression model
    model = LinearRegression()

    # Train the model on the entire dataset
    model.fit(X, y)
    
    return model

In [227]:
def score_piecewise_regression_model(df, model, position, season):
    df_subset = df[(df['position'] == position) & (df['season'] == season)]

    # Specify the feature columns explicitly, including GK
    feature_columns = [
        'value_first_gw', 
        'team_strength']

    # Step 2: Split the data into features (X) and target (y)
    X = df_subset[feature_columns]


    # Train the model on the entire dataset
    y_pred = model.predict(X)

    #  Add predictions as a new column in df_subset
    df_subset['predicted_total_score'] = y_pred

    
    return df_subset

In [193]:
# Load data

# Step 1: Create a list of season years to iterate through
season_years = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']

# Step 2: Initialize an empty list to store the DataFrames
data_frames = []

# Step 3: Loop through the season years and read each CSV file using f-strings
for season in season_years:
    file_path = f'data/fpl_data/{season}.csv' 
    df = pd.read_csv(file_path) 
    data_frames.append(df) 

# Step 4: Concatenate all DataFrames into a single DataFrame
df = pd.concat(data_frames, ignore_index=True)

In [241]:
# Create dictionary to save model artifacts
model_dict = {}

# Initialize an empty list to store DataFrames
dataframes = []

seasons = ['2019-20','2020-21','2021-22','2022-23','2023-24']
positions = ['GK','DEF','MID','FWD']

for season in seasons:
    for position in positions:

        # Get season start year
        season_start = int(season[:4])
        
        # Create model
        model_name = f'{season}_{position}'
        model = create_piecewise_regression_model(df=df, position=position, season=season)
        model_dict[model_name]=model

        # Score model on subsequent season
        if season == seasons[0]:
            continue
        else:
            previous_season = f"{season_start-1}-{str(season_start)[-2:]}" 
            previous_model_name = f'{previous_season}_{position}'
            model_prev = model_dict[previous_model_name]

            df_subset = score_piecewise_regression_model(df=df, model=model_prev, position=position, season=season)

            # Append the df_subset to the list
            dataframes.append(df_subset)

# Join scored data
df_scored = pd.concat(dataframes, ignore_index=True)

            


C:\Users\edwar\AppData\Local\Temp\ipykernel_9900\1092402770.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['predicted_total_score'] = y_pred
C:\Users\edwar\AppData\Local\Temp\ipykernel_9900\1092402770.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['predicted_total_score'] = y_pred
C:\Users\edwar\AppData\Local\Temp\ipykernel_9900\1092402770.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [261]:
# Modulise
# Create streamlit plot
# Calculate MAE etc.
# Predict with championship data

In [259]:
# # Step 1: Set the style of seaborn
# sns.set(style="whitegrid")

# # Step 2: Create a figure to hold multiple plots
# plt.figure(figsize=(12, 8))

# # Get unique positions
# unique_positions = df_scored['position'].unique()

# # Create a scatter plot for each unique position
# for position in unique_positions:
#     # Filter the DataFrame for the current position
#     df_position = df_scored[df_scored['position'] == position]
    
#     # Create scatter plot
#     scatter = sns.scatterplot(
#         data=df_position,
#         x='total_points',
#         y='predicted_total_score',
#         hue='promoted_from_championship',  # Color code by the binary flag
#         style='promoted_from_championship',  # Different markers for different categories
#         markers={0: 'o', 1: 'X'},  # Circle for 0, X for 1
#         palette={0: 'blue', 1: 'orange'},  # Color mapping
#         s=100  # Size of the markers
#     )
    
#     # Add labels for each point
#     for i in range(len(df_position)):
#         plt.text(
#             df_position['total_points'].iloc[i],
#             df_position['predicted_total_score'].iloc[i],
#             df_position['name'].iloc[i],
#             fontsize=9,  # Font size for the labels
#             ha='right'  # Horizontal alignment
#         )

# # Step 3: Customize the plot
# plt.title('Scatter Plot of Total Score vs. Predicted Total Score by Position')
# plt.xlabel('Total Score')
# plt.ylabel('Predicted Total Score')
# plt.legend(title='Promoted from Championship', loc='upper left')
# plt.grid(True)

# # Step 4: Show the plot
# plt.show()


In [260]:
# df_scored[df_scored['name']=='Erling Haaland'].T